In [1]:
"""
from keras import models
from keras import layers
from keras.preprocessing import image
from keras import optimizers
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
"""

import numpy as np
import os

from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.preprocessing import image
import tensorflow as tf
#from tensorflow.python import keras

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras.applications import VGG16
from tensorflow.python.keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
conv_base1 = VGGFace(include_top=False,model = 'resnet50', weights = 'vggface',input_shape=(150,150,3))
conv_base2=VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
#conv_base2.summary()

ValueError: Input size must be at least 197x197; got `input_shape=(150, 150, 3)`

In [3]:

# 在conv_base的基础上添加全连接分类网络
model = models.Sequential()
#model.add(conv_base)
#conv_base=VGGFace(include_top=False,model = 'vgg16', weights = 'vggface',input_shape=(150,150,3))
model.add(conv_base2)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
conv_base2.trainable = False
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc'])

#model.summary()

Keras model to TensorFlow Estimator

In [4]:
model_dir = os.path.join(os.getcwd(), "models/vgg16_2FC_TF")
os.makedirs(model_dir, exist_ok=True)
print("model_dir: ",model_dir)
vgg16_classifier = tf.keras.estimator.model_to_estimator(keras_model=model,
                                                    model_dir=model_dir)

model_dir:  /home/qsc/DEMO/VGGFace_TF/models/vgg16_2FC_TF
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/home/qsc/DEMO/VGGFace_TF/models/vgg16_2FC_TF', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4b44e1358>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
def load_file(examples_list_file):
    lines = np.genfromtxt(examples_list_file, delimiter="", dtype=[('col1', 'S120'), ('col2', 'i8')])
    #f=open(train_file)
    #lines=f.readlines()
    examples = []
    labels = []
    for example, label in lines:
        examples.append(example)
        labels.append(label)
    return np.asarray(examples), np.asarray(labels)

In [10]:
def imgs_input_fn(filenames, labels=None, perform_shuffle=False, repeat_count=1, batch_size=1):
    def _parse_function(filename, label):
        image_string = tf.read_file(filename)
        image = tf.image.decode_image(image_string, channels=3)
        image.set_shape([None, None, None])
        image = tf.image.resize_images(image, [150, 150])
        #image = tf.subtract(image, 116.779) # Zero-center by mean pixel
        image.set_shape([150, 150, 3])
        #image = tf.reverse(image, axis=[2]) # 'RGB'->'BGR'
        d = dict(zip(["vgg16_input"], [image])), label
        return d
    if labels is None:
        labels = [0]*len(filenames)
    labels=np.array(labels)
    # Expand the shape of "labels" if necessory
    if len(labels.shape) == 1:
        labels = np.expand_dims(labels, axis=1)
    filenames = tf.constant(filenames)
    labels = tf.constant(labels)
    labels = tf.cast(labels, tf.float32)
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(_parse_function)
    if perform_shuffle:
        # Randomizes input using a window of 256 elements (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count)  # Repeats dataset this # times
    dataset = dataset.batch(batch_size)  # Batch size to use
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

In [11]:
base_dir = 'k-fold'
train_files=base_dir+"/path_txt/train06.txt"
valid_files=base_dir+"/path_txt/valid06.txt"
path_tfrecords_train="k-fold/tfrecords/train11.tfrecords"
path_tfrecords_valid="k-fold/tfrecords/test11.tfrecords"

In [12]:

"""
next_batch = imgs_input_fn(examples_train01,labels_train01, perform_shuffle=True, batch_size=20)
with tf.Session() as sess:
    first_batch = sess.run(next_batch)
x_d = first_batch[0]['input_1']

print(x_d.shape)
img = image.array_to_img(x_d[8])
img
len(examples_train01)

"""


"\nnext_batch = imgs_input_fn(examples_train01,labels_train01, perform_shuffle=True, batch_size=20)\nwith tf.Session() as sess:\n    first_batch = sess.run(next_batch)\nx_d = first_batch[0]['input_1']\n\nprint(x_d.shape)\nimg = image.array_to_img(x_d[8])\nimg\nlen(examples_train01)\n\n"

In [13]:
examples_train01,labels_train01=load_file(train_files)
examples_valid01,labels_valid01=load_file(valid_files)
"""
next_batch = imgs_input_fn(examples_valid01, labels=labels_valid01, perform_shuffle=True, batch_size=30)
#for i in examples_train01[:10]:
    #print (i)
with tf.Session() as sess:
    first_batch = sess.run(next_batch)
x_d = first_batch[0]['vgg16_input']
x_d_label=first_batch[1]

print(x_d.shape)
print(x_d_label)
img = image.array_to_img(x_d[8])
img

"""


"\nnext_batch = imgs_input_fn(examples_valid01, labels=labels_valid01, perform_shuffle=True, batch_size=30)\n#for i in examples_train01[:10]:\n    #print (i)\nwith tf.Session() as sess:\n    first_batch = sess.run(next_batch)\nx_d = first_batch[0]['vgg16_input']\nx_d_label=first_batch[1]\n\nprint(x_d.shape)\nprint(x_d_label)\nimg = image.array_to_img(x_d[8])\nimg\n\n"

In [14]:
vgg16_classifier.train(input_fn=lambda:imgs_input_fn(examples_train01,
                                                    labels=labels_train01,
                                                    perform_shuffle=True,
                                                    repeat_count=2,
                                                     batch_size=20))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_2FC_TF/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_2FC_TF/model.ckpt.
INFO:tensorflow:loss = 62.7032, step = 2
INFO:tensorflow:global_step/sec: 4.14506
INFO:tensorflow:loss = 13.9487, step = 101 (24.126 sec)
INFO:tensorflow:global_step/sec: 4.47793
INFO:tensorflow:loss = 11.4533, step = 201 (22.332 sec)
INFO:tensorflow:global_step/sec: 4.2921
INFO:tensorflow:loss = 8.88362, step = 301 (23.299 sec)
INFO:tensorflow:global_step/sec: 4.30494
INFO:tensorflow:loss = 17.3326, step = 401 (23.229 sec)
INFO:tensorflow:global_step/sec: 4.19658
INFO:tensorflow:loss = 26.1787, step = 501 (23.829 sec)
INFO:tensorflow:global_step/sec: 

In [15]:
evaluate_results = vgg16_classifier.evaluate(
    input_fn=lambda: imgs_input_fn(examples_valid01, 
                                   labels=labels_valid01, 
                                   perform_shuffle=False,
                                   batch_size=1))
print("Evaluation results")
for key in evaluate_results:
    print("   {}, was: {}".format(key, evaluate_results[key]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-21-08:28:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_2FC_TF/model.ckpt-1649
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-21-08:28:17
INFO:tensorflow:Saving dict for global step 1649: acc = 0.0, global_step = 1649, loss = 27.4714
Evaluation results
   acc, was: 0.0
   loss, was: 27.47136878967285
   global_step, was: 1649


In [41]:
predict_results = vgg16_classifier.predict(
    input_fn=lambda: imgs_input_fn(examples_valid01[:10], 
                                   labels=None, 
                                   perform_shuffle=False,
                                   batch_size=10))
for i in examples_valid01[:10]:
    print(i)
predict_logits = []
for prediction in predict_results:
    predict_logits.append(prediction['dense_2'][0])
    print(prediction['dense_2'][0])

b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img116.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img117.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img118.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img119.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img120.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img121.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img122.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img123.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img124.jpg'
b'/home/qsc/DEMO/CASME2_RAW_selected/sub26/EP03_10/img125.jpg'
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt-799
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
2.31939e-33
5.06815e-34
1.08199e-33
3.46221e-33
1.40334e-33
2.62148e-3

In [19]:
acclist=[]
for i in range(26):
    print("*******round%02d**********"%(i+1))
    base_dir = 'k-fold'
    train_files=base_dir+"/path_txt/train%02d.txt"%(i+1)
    valid_files=base_dir+"/path_txt/valid%02d.txt"%(i+1)
    #path_tfrecords_train="k-fold/tfrecords/train%02d.tfrecords"%(i+1)
    #path_tfrecords_valid="k-fold/tfrecords/test%02d.tfrecords"%(i+1)
    examples_train,labels_train=load_file(train_files)
    examples_valid,labels_valid=load_file(valid_files)
    vgg16_classifier.train(input_fn=lambda:imgs_input_fn(examples_train,
                                                    labels=labels_train,
                                                    perform_shuffle=True,
                                                    repeat_count=1,
                                                     batch_size=20))
    
    evaluate_results = vgg16_classifier.evaluate(
    input_fn=lambda: imgs_input_fn(examples_valid, 
                                   labels=labels_valid, 
                                   perform_shuffle=False,
                                   batch_size=1))
    print("Evaluation results")
    for key in evaluate_results:
        print("   {}, was: {}".format(key, evaluate_results[key]))
        acclist.append(evaluate_results[key])
    
print(acclist)    
    
    

*******round01**********
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 24.1814, step = 0
INFO:tensorflow:global_step/sec: 4.68479
INFO:tensorflow:loss = 29.7985, step = 100 (21.346 sec)
INFO:tensorflow:global_step/sec: 4.91342
INFO:tensorflow:loss = 21.3275, step = 200 (20.352 sec)
INFO:tensorflow:global_step/sec: 4.82957
INFO:tensorflow:loss = 8.04729, step = 300 (20.706 sec)
INFO:tensorflow:global_step/sec: 4.88593
INFO:tensorflow:loss = 22.1299, step = 400 (20.467 sec)
INFO:tensorflow:global_step/sec: 4.81122
INFO:tensorflow:loss = 24.9463, step = 500 (20.785 sec)
INFO:tensorflow:global_step/sec: 4.79157
INFO:tensorflow:loss = 24.9463, step = 600 (20.870 sec)
INFO:te

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt-3233
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3234 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 12.8755, step = 3233
INFO:tensorflow:global_step/sec: 4.42198
INFO:tensorflow:loss = 16.8991, step = 3333 (22.615 sec)
INFO:tensorflow:global_step/sec: 4.76049
INFO:tensorflow:loss = 26.9581, step = 3433 (21.006 sec)
INFO:tensorflow:global_step/sec: 4.64986
INFO:tensorflow:loss = 10.059, step = 3533 (21.506 sec)
INFO:tensorflow:global_step/sec: 4.74097
INFO:tensorflow:loss = 29.3722, step = 3633 (21.092 sec)
INFO:tensorflow:global_step/sec: 4.65087
INFO:tensorflow:loss = 17.3015, step = 3733 (21.501 sec)
INFO:tensorflow:global_step/sec: 4.63247
INFO:tensorflow:loss = 10.059, step = 3833 

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt-6467
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 6468 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 11.6684, step = 6467
INFO:tensorflow:global_step/sec: 4.39625
INFO:tensorflow:loss = 16.0944, step = 6567 (22.747 sec)
INFO:tensorflow:global_step/sec: 4.51458
INFO:tensorflow:loss = 17.3015, step = 6667 (22.151 sec)
INFO:tensorflow:global_step/sec: 4.55212
INFO:tensorflow:loss = 10.8637, step = 6767 (21.968 sec)
INFO:tensorflow:global_step/sec: 4.63181
INFO:tensorflow:loss = 26.1534, step = 6867 (21.590 sec)
INFO:tensorflow:global_step/sec: 4.60022
INFO:tensorflow:loss = 18.5085, step = 6967 (21.738 sec)
INFO:tensorflow:global_step/sec: 4.51779
INF

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9631 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 15.2897, step = 9630
INFO:tensorflow:global_step/sec: 4.4957
INFO:tensorflow:loss = 12.8755, step = 9730 (22.244 sec)
INFO:tensorflow:global_step/sec: 4.7123
INFO:tensorflow:loss = 15.692, step = 9830 (21.221 sec)
INFO:tensorflow:global_step/sec: 4.54344
INFO:tensorflow:loss = 8.04719, step = 9930 (22.010 sec)
INFO:tensorflow:global_step/sec: 4.59824
INFO:tensorflow:loss = 30.9817, step = 10030 (21.747 sec)
INFO:tensorflow:global_step/sec: 4.6145
INFO:tensorflow:loss = 23.7392, step = 10130 (21.671 sec)
INFO:tensorflow:global_step/sec: 4.59242
INFO:tensorflow:loss = 20.118, step = 10230 (21.775 sec)
INFO:tensorflow:global_step/sec: 4.60977
INFO:tensorflow:loss = 16.0944, step = 10330 (21.693 sec)
INFO:tensorflow:global_step/sec: 4.51839
INFO:tensorflow:loss = 18.5085, step

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt-12918
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 12919 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 11.6684, step = 12918
INFO:tensorflow:global_step/sec: 4.24962
INFO:tensorflow:loss = 14.4849, step = 13018 (23.532 sec)
INFO:tensorflow:global_step/sec: 4.43668
INFO:tensorflow:loss = 16.0944, step = 13118 (22.539 sec)
INFO:tensorflow:global_step/sec: 4.45601
INFO:tensorflow:loss = 10.8637, step = 13218 (22.441 sec)
INFO:tensorflow:global_step/sec: 4.41782
INFO:tensorflow:loss = 31.384, step = 13318 (22.636 sec)
INFO:tensorflow:global_step/sec: 4.3615
INFO:tensorflow:loss = 20.118, step = 13418 (22.928 sec)
INFO:tensorflow:global_step/sec: 4.3926

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt-16072
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 16073 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 9.25427, step = 16072
INFO:tensorflow:global_step/sec: 4.49812
INFO:tensorflow:loss = 13.6802, step = 16172 (22.232 sec)
INFO:tensorflow:global_step/sec: 4.6528
INFO:tensorflow:loss = 14.4849, step = 16272 (21.493 sec)
INFO:tensorflow:global_step/sec: 4.64102
INFO:tensorflow:loss = 9.65663, step = 16372 (21.547 sec)
INFO:tensorflow:global_step/sec: 4.63041
INFO:tensorflow:loss = 25.751, step = 16472 (21.596 sec)
INFO:tensorflow:global_step/sec: 4.63965
INFO:tensorflow:loss = 26.5557, step = 16572 (21.553 sec)
INFO

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt-19344
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 19345 into /home/qsc/DEMO/VGGFace_TF/models/vgg16_classifier/model.ckpt.
INFO:tensorflow:loss = 11.6684, step = 19344
INFO:tensorflow:global_step/sec: 4.50047
INFO:tensorflow:loss = 14.0826, step = 19444 (22.221 sec)
INFO:tensorflow:global_step/sec: 4.58525
INFO:tensorflow:loss = 18.5085, step = 19544 (21.809 sec)
INFO:tensorflow:global_step/sec: 4.69169
INFO:tensorflow:loss = 8.04719, step = 19644 (21.314 sec)
INFO:tensorflow:global_step/sec: 4.61768
INFO:tensorflow:loss = 30.177, step = 19744 (21.656 sec)
INFO:tensorflow:global_step/sec: 4.65421
INFO:tensorflow:loss = 23.7392, step = 19844 (21.486 sec)
INF

In [21]:
ave=acclist[::3]
#ave

np.mean(ave)

0.11319575

In [13]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda: imgs_input_fn(examples_train01,
                                                                   labels=labels_train01,
                                                                   perform_shuffle=True,
                                                                   repeat_count=5,
                                                                   batch_size=20), 
                                    max_steps=500)
eval_spec = tf.estimator.EvalSpec(input_fn=lambda: imgs_input_fn(examples_valid01,
                                                                 labels=labels_valid01,
                                                                 perform_shuffle=False,
                                                                 batch_size=1))

tf.estimator.train_and_evaluate(vggface_classifier, train_spec, eval_spec)

NameError: name 'vggface_classifier' is not defined